# Gemini + Chroma + Cohereで RAG を作ってみる

**RAG**: embedding(Gemini) - similarity retrieval(Chroma) - re-ranking(Cohere)  

Langchainを使わないで、なるべくプリミティブなAPIで
 - ennbedding(ベクトル化)/LLMは、Gemini ennbedding/LLMモデルを直接使用
 - ベクトルDBは、ChromaDB を直接使用
 - reranikgは、Cohere rerank を直接使用

In [87]:
import os
import google.genai as genai
import chromadb
import cohere
print(f"genai={genai.__version__}, chroma={chromadb.__version__}, cohere={cohere.__version__}")

# ====== Gemini の設定 ======
# Gemini API Keyを取得
with open('GOOGLE_API_KEY.txt', 'r') as f:  # ファイルからAPI Keyを取得
    api_key = f.read().strip()
# Geminiモデルを指定
llm_model       = 'gemini-2.0-flash'
embedding_model = 'gemini-embedding-001'
# Geminiクライアントの作成
gemini_client = genai.Client(api_key=api_key)

# ====== ChromaDB の設定 ======
# clientの作成
# 　メモリ上だけ (永続化しない), 
# 　永続化させる場合には、chromadb.PersistentClient(path='save_to')を使用)
chroma_client = client = chromadb.EphemeralClient()

# ====== Cohere の設定 ======
# Cohereクライアントを作成
with open('Cohere_API_KEY.txt', 'r') as f:  # ファイルからアクセスキーを取得
    api_key = f.read().strip()
co = cohere.ClientV2(api_key=api_key)

genai=1.45.0, chroma=1.2.1, cohere=5.19.0


## まずは簡単なテキストで

In [2]:
# 検索対象テキスト ======================
texts = [
    '石川です。エンジニアです。散歩は嫌いではありません。', 
    '果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。',
    '佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。',
    '東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。',
    'サイクリングの好きな鈴木さんは自転車で会社へ通勤しています。高橋さんは会社の同僚です。',
    '伊藤さんと山本さんは、よく一緒に奥多摩へキャンプに行くようです',
    '中村さんは高橋さんの勤めている会社の上司です。よく飲みに誘われますが参加するかは半々です。',
    '京都にお住いの小林さんは、佐藤さんのおいです。',
    '加藤さんは5人家族です。',
]

# テキストをエンベディング化(vector化)する  ======================
# --- Embeddingの取得 ---
response = gemini_client.models.embed_content(
    model=embedding_model,
    contents=texts
)
embeddings = [e.values for e in response.embeddings] # 配列へ変換

print(f"len(embeddings)={len(embeddings)}") # 内容確認:
for i, embedding in enumerate(embeddings[:2]):
    print(f"{i:2d} embedding First 5 values: {embedding[:5]}")

# ChromaDBの初期化、ローディング  ======================
# DB内collectionの初期化(既存のcollectionがあったら削除)
collection_name = 'novels'   # 任意の文字列
try:
    chroma_client.delete_collection(collection_name)
except:
    pass
collection = chroma_client.create_collection(collection_name)

# Document から必要な情報をまとめる
ids = [f"doc{i}" for i in range(len(texts))] # id: ユニークな文字列

# 確認
print(f"Confirm: len(ids)={len(ids)}, len(embeddings)={len(embeddings)}, len(texts)={len(texts)}")

# 一括で追加
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=texts,
#    metadatas=metadatas        # (今回未使用、空辞書だとエラー)
)
print(f"collection count={collection.count()}")

# LLMモデルを使用してレスポンスを生成
queries = []
queries.append('山田さんの職業は何ですか。')
queries.append('佐藤さんの職業は何ですか。')
queries.append('お酒を飲む人は誰ですか。')
queries.append('関東に住んでいる人は誰ですか。')
print(f"\nQ&A ----------------")
for q_no, query in enumerate(queries):

    # query を embedding
    embedded_query = gemini_client.models.embed_content(
        model=embedding_model,
        contents=query
    )
    query_vector  = embedded_query.embeddings[0].values

    # embeddingでretrieve
    results = collection.query(
        query_embeddings=[query_vector ],
        n_results=3
    )
    retreaved_texts = results['documents'][0]

    # Reraning <<略>>

    # retreaveされたtextsで、LLMを使ってqueryに回答
    response = gemini_client.models.generate_content(
        model=llm_model,
        contents=[f"QUESTION{query}", f"CONTENTS"] + retreaved_texts,
        config=genai.types.GenerateContentConfig(
            system_instruction="あなたは気さくな隣人です。質問に日常会話的に答えてください。",
            temperature=0.7,  # ★ creativity の度合いを調整
        )
    )

    # 結果表示
    print(f"\nQ{q_no}: query={query},\nresponse={response.text}")
    for i in range(len(results['documents'][0])):     # [0]が必要なのは、複数のqueryを投げられるため
        print(f"retrieved doc {i}: id={results['ids'][0][i]}, " +
              f"distances={results['distances'][0][i]}, metadata={results['metadatas'][0][i]}" +
              f"\n          text={results['documents'][0][i][:200]}")   # 最初の200文字だけ表示


len(embeddings)=9
 0 embedding First 5 values: [0.0010123871, -0.027725141, 0.004763818, -0.085715435, -0.016764862]
 1 embedding First 5 values: [-0.0032862844, -0.019554267, 0.006109047, -0.07946284, 0.008544021]
Confirm: len(ids)=9, len(embeddings)=9, len(texts)=9
collection count=9

Q&A ----------------

Q0: query=山田さんの職業は何ですか。,
response=あら、山田さんですか。山田さんは果物屋さんですよ。前は八百屋さんだったみたいですね。甘いものがお好きなんだとか。意外とお酒は飲めないらしいですよ。

retrieved doc 0: id=doc1, distances=0.4490359425544739, metadata=None
          text=果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。
retrieved doc 1: id=doc4, distances=0.732514500617981, metadata=None
          text=サイクリングの好きな鈴木さんは自転車で会社へ通勤しています。高橋さんは会社の同僚です。
retrieved doc 2: id=doc2, distances=0.743468165397644, metadata=None
          text=佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。

Q1: query=佐藤さんの職業は何ですか。,
response=あら、佐藤さんのことですか。佐藤さんはもともと酒屋さんをやってたみたいだけど、今はコンビニの店長さんなんですね！しかも、泣き上戸だって(笑)。お酒の席で会ったら、ちょっと面白い話が聞けそうですね！

retrieved doc 0: id=doc2, distances=0.40844571590423584, metadata=None
      

## 大きなドキュメントの分割、メタデータ利用

ドキュメントとして、青空文庫を利用させていただきました。

一部Langchainを使用  
以下の機能は単機能で、特にRecursiveCharacterTextSplitter()は代わりになるライブラリが見つからず、スクラッチで作成するのも結構大変ということで、使わせていただきます。
 - Documentクラス                    # テキスト+メタデータ  
 - DirectoryLoader()                 # ディレクトリ単位でファイルを取得、Document型へ  
 - RecursiveCharacterTextSplitter()  # Document型のテキストを指定サイズでChankへ分割  
                                     # 区切文字列、ラップサイズを指定、メタデータを各Chankへコピー  

### Vector Storeへの登録

In [93]:
from langchain.schema import Document
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# テキストファイル・メタデータ
matadata_dic = {
    '1_kumono_ito.txt': {
        'genre': 'novel', 'title':'蜘蛛の糸',          'author':'芥川竜之介', 'year':'1918'},
    '2_chumonno_oi_ryoriten.txt': {
        'genre': 'novel', 'title':'注文の多い料理店',  'author':'宮沢賢治',   'year':'1924'},
    '2_kazeno_matasaburo.txt': {
        'genre': 'novel', 'title':'風の又三郎',        'author':'宮沢賢治',   'year':'1934'},
    '2_serohikino_goshu.txt': {
        'genre': 'novel', 'title':'セロ弾きのゴーシュ', 'author':'宮沢賢治',   'year':'1934'},
    '2_gingatetsudono_yoru.txt': {
        'genre': 'novel', 'title':'銀河鉄道の夜',      'author':'宮沢賢治',   'year':'1934'},
    '3_bocchan.txt': {
        'genre': 'novel', 'title':'坊ちゃん',         'author':'夏目漱石',   'year':'1906'},
    '4_kaijin_nijumenso.txt': {
        'genre': 'novel', 'title':'怪人二十面相',      'author':'江戸川乱歩', 'year':'1936'},
    '5_sanshodayu.txt': {
        'genre': 'novel', 'title':'山椒大夫',         'author':'森鷗外',     'year':'1915'},
}
null_dic = {'genre':'','title':'', 'author':'','year':''}

# 小説データをロード ======================
novels_dir = './novels/'
loader = DirectoryLoader(novels_dir, glob='*.txt') # ディレクトリ内の.txtを指定して
documents = loader.load()   # ドキュメント(メタデータ(ファイル名)+テキスト)としてロード

# 各ドキュメントにメタデータを付与 ======================
for doc in documents:
    fn = os.path.basename(doc.metadata['source'])
    doc.metadata.update({'filename': fn} | matadata_dic.get(fn, null_dic))
print(documents[0].metadata) # 確認

# ChromaDBの初期化、ローディング  ======================
# DB内collectionの初期化(既存のcollectionがあったら削除)
collection_name = 'novels'   # 任意の文字列
try:
    chroma_client.delete_collection(collection_name)
except:
    pass
collection = chroma_client.create_collection(collection_name)

# ドキュメント毎に (embedding tokenサイズ制限により) ======================
for doc_no, doc in enumerate(documents):
    # ドキュメントをチャンクへ分割 ======================
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 10000,
        chunk_overlap = 100,
        separators = ['。', '\n'] # 分割位置の指定
    )
    chunks = text_splitter.split_documents([doc])
    print(f"{doc_no}: {doc.metadata['title']}, n_chunks: {len(chunks)}")                                            # 確認
    #print(f"\nchunks[0] --------\n{chunks[0].page_content[:100]},\n{chunks[0].metadata}")
    
    # chunk を embedding  ======================
    embedded_chunks = gemini_client.models.embed_content(
        model = embedding_model,
        contents = [c.page_content for c in chunks]
    )
    chunk_vectors = [e.values for e in embedded_chunks.embeddings]
        
    # id生成
    ids = [f"doc{doc_no}-{i}" for i in range(len(chunks))] # id: ユニークな文字列
    # 一括で追加
    collection.add(
        ids = ids,
        embeddings = chunk_vectors,
        documents = [c.page_content for c in chunks],
        metadatas = [m.metadata for m in chunks]
    )
    print(f"collection count={collection.count()}")

{'source': 'novels\\1_kumono_ito.txt', 'filename': '1_kumono_ito.txt', 'genre': 'novel', 'title': '蜘蛛の糸', 'author': '芥川竜之介', 'year': '1918'}
0: 蜘蛛の糸, n_chunks: 1
collection count=1
1: 注文の多い料理店, n_chunks: 1
collection count=2
2: 銀河鉄道の夜, n_chunks: 6
collection count=8
3: 風の又三郎, n_chunks: 4
collection count=12
4: セロ弾きのゴーシュ, n_chunks: 2
collection count=14
5: 坊ちゃん, n_chunks: 11
collection count=25
6: 怪人二十面相, n_chunks: 12
collection count=37
7: 山椒大夫, n_chunks: 3
collection count=40
8: , n_chunks: 10
collection count=50


### Retreival, Re-ranking, LLM Response Generation

In [97]:
# LLMモデルを使用してレスポンスを生成
queries = []
queries.append('蜘蛛の糸は何に使われましたか')
queries.append('ゴーシュの仕事は何ですか')
queries.append('坊ちゃんの姓名は何ですか')
queries.append('怪人の名前は何ですか')
queries.append('二十面相が現れた場所をすべて挙げてください')
queries.append('山椒大夫は最後にどうなりましたか')

print(f"\nQ&A ----------------")
top_k = 5
for q_no, query in enumerate(queries):
    print(f"\nQ{q_no}: query={query} ==============")
    
    # Embed the query
    embedded_query = gemini_client.models.embed_content(
        model=embedding_model,
        contents=query
    )
    query_vector = embedded_query.embeddings[0].values

    # embeddingでretrieve
    retreaved_results = collection.query(
        query_embeddings = [query_vector ],
        n_results = top_k*2
    )
    retreaved_texts = retreaved_results['documents'][0]
    # 結果表示
    print(f"Retrieval (Chrom distances: 値が小さい方が近い) 結果:")
    for rank, doc in enumerate(retreaved_results['documents'][0]): # [0]が必要なのは、複数のqueryを投げられるため
        print(f"retrieved chunk {rank}: " + 
              f"id={retreaved_results['ids'][0][rank]}, " +
              f"distance={retreaved_results['distances'][0][rank]:.5f}, " +
              f"title={retreaved_results['metadatas'][0][rank]['title']}")

    # Rerank the documents
    candidates = retreaved_results['documents'][0]
    rerank_results = co.rerank(
        model="rerank-v3.5",
        query=query,
        documents=candidates,
        top_n=top_k,
    ).results
    reranked_texts  = [candidates[r.index] for r in rerank_results]
    # --- 結果表示 ---
    print(f"Retrieval + Rerank (Cohere reranking score: 値が大きい方が近似) 結果:")
    for rank, result in enumerate(rerank_results): # index: 元のDocumentリストへのindex
        print(f"reranked chunk {rank}: " + 
              f"id={retreaved_results['ids'][0][result.index]}, " +
              f"score={result.relevance_score:.5f}, " +
              f"title={retreaved_results['metadatas'][0][result.index]['title']}")

    # retreaveされたtextsで、LLMを使ってqueryに回答
    response = gemini_client.models.generate_content(
        model=llm_model,
        contents=[f"QUESTION{query}", f"CONTENTS"] + reranked_texts, # rerank結果を利用
        config=genai.types.GenerateContentConfig(
            system_instruction="あなたは文学者です。質問に誠実にに答えてください。",
            temperature=0.7,  # ★ creativity の度合いを調整
        )
    )
    # 結果表示
    print(f"\nQ{q_no}: query={query},\nresponse={response.text}")



Q&A ----------------

Q0: query=蜘蛛の糸は何に使われましたか ==============
Retrieval (Chrom distances: 値が小さい方が近い) 結果:
retrieved chunk 0: id=doc0-0, distance=0.59081, title=蜘蛛の糸
retrieved chunk 1: id=doc6-4, distance=0.79448, title=怪人二十面相
retrieved chunk 2: id=doc6-5, distance=0.80821, title=怪人二十面相
retrieved chunk 3: id=doc2-4, distance=0.81376, title=銀河鉄道の夜
retrieved chunk 4: id=doc6-3, distance=0.81417, title=怪人二十面相
retrieved chunk 5: id=doc6-2, distance=0.81449, title=怪人二十面相
retrieved chunk 6: id=doc6-9, distance=0.81793, title=怪人二十面相
retrieved chunk 7: id=doc3-2, distance=0.82108, title=風の又三郎
retrieved chunk 8: id=doc6-6, distance=0.82363, title=怪人二十面相
retrieved chunk 9: id=doc2-2, distance=0.82711, title=銀河鉄道の夜
Retrieval + Rerank (Cohere reranking score: 値が大きい方が近似) 結果:
reranked chunk 0: id=doc0-0, score=0.70165, title=蜘蛛の糸
reranked chunk 1: id=doc3-2, score=0.46643, title=風の又三郎
reranked chunk 2: id=doc6-5, score=0.39754, title=怪人二十面相
reranked chunk 3: id=doc6-6, score=0.26186, title=怪人二十面相
rer